In [ ]:
import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

import networkx as nx
import scipy
from scipy.spatial.distance import cdist
from scipy.linalg import eigh
from scipy.sparse import linalg
from scipy import sparse
import sklearn
from sklearn.cluster import Birch, KMeans, MeanShift, DBSCAN, SpectralClustering
from sklearn.decomposition import TruncatedSVD
from normalized_cut import normalized_cut

from point_cloud_utils import get_pcd
from aggregate_pointcloud import aggregate_pointcloud
from reproject_merged_pointcloud import reproject_points_to_label, merge_associations

Here we define the dataset depending on kitti sequence!

In [ ]:
DATASET_PATH = os.path.join('/Users/laurenzheidrich/Downloads/','fused_dataset')
SEQUENCE_NUM = 7

config_filtered = KittiOdometryDatasetConfig(
    cache=True,
    dataset_path=DATASET_PATH,
    correct_scan_calibration=True,
    filters=FilterList(
        [
            KittiGTMovingObjectFilter(
                os.path.join(
                    DATASET_PATH,
                    "sequences",
                    "%.2d" % SEQUENCE_NUM,
                    "labels",
                )
            ),
            RangeFilter(2.5, 120),
            ApplyPose(),
        ]
    ),
)

dataset = KittiOdometryDataset(config_filtered, SEQUENCE_NUM)

Now we read in the point cloud and the left and right image of the stereo camera. If labels for those images are available they can be read in, too!

In [ ]:
ind_start = 58
ind_end = 65

pcd_aggregated, T_pcd = aggregate_pointcloud(dataset, ind_start, ind_end, clip_to_imageframe=True)

pcd_cut = pcd_aggregated
pcd_cut = pcd_cut.select_by_index(np.where(np.asarray(pcd_cut.points)[:,0] > 6)[0])
pcd_cut = pcd_cut.select_by_index(np.where(np.asarray(pcd_cut.points)[:,0] < 15)[0])

pcd = pcd_cut.voxel_down_sample(voxel_size=0.25)

# Just use a subset of pcd for testing
#pcd = pcd.select_by_index(np.where(np.asarray(pcd.points)[:,0] < 10)[0])
#pcd = pcd.select_by_index(np.where(np.asarray(pcd.points)[:,0] > 6)[0])
#pcd.points = pcd.points[::10]
num_points = np.asarray(pcd.points).shape[0]
print("num points: ", num_points)

In [ ]:
#o3d.io.write_point_cloud("pcd_cropped.pcd", pcd, write_ascii=False, compressed=False, print_progress=False)

No we can to the point to label reprojection for each timestep and merge them in a list

In [ ]:
cams = ["cam2", "cam3"]
cam_ind = 0
point_to_label_reprojections = []

for points_index in range(ind_start, ind_end):
	label_PIL = dataset.get_sam_label(cams[cam_ind], points_index)
	label = cv2.cvtColor(np.array(label_PIL), cv2.COLOR_RGB2BGR)

	T_world2lidar = np.linalg.inv(dataset.get_pose(points_index))
	T_lidar2cam, K = dataset.get_calibration_matrices(cams[cam_ind])
	T_world2cam = T_lidar2cam @ T_world2lidar
	
	point_to_label_reprojections.append(reproject_points_to_label(np.array(pcd.points), T_pcd, label, T_world2cam, K, hidden_point_removal=True))

From the merged list we can now build the association matrix

In [ ]:
association_matrix = merge_associations(point_to_label_reprojections, len(pcd.points))

Here we define important parameters and build the point-to-point distance matrix

In [ ]:
proximity_threshold = 1 # meters that points can be apart froim each other and still be considered neighbors
alpha = 6.0 # weight of the spatial proximity term
beta = 1.0 # weight of the feature similarity term
points = np.asarray(pcd.points)
dist_matrix = cdist(points, points)

Now we can build the graph with nodes and edges!

In [ ]:
mask = np.where(dist_matrix <= proximity_threshold, 1, 0)

feature_diff = np.zeros((num_points, num_points))

for i in range(num_points):
    for j in range(i+1, num_points):
        if mask[i,j] == 1:
            feature_diff[i,j] = np.exp(-beta * np.sum(association_matrix[i] != association_matrix[j]))
            feature_diff[j,i] = feature_diff[i,j]

In [ ]:
A = np.exp(-alpha * (mask * dist_matrix)) * feature_diff
G = nx.from_numpy_array(A)

In [ ]:

G = nx.Graph()

for i in range(num_points):
    G.add_node(i, pos=points[i], label=0, associations=association_matrix[i])

for i in range(num_points):
    for j in range (i+1, num_points):
        if dist_matrix[i, j] <= proximity_threshold:
            dist_weight = np.exp(-alpha * dist_matrix[i, j])
            feature_weight = np.exp(-beta * np.sum(G.nodes[i]['associations'] != G.nodes[j]['associations']))
            G.add_edge(i, j, weight=(dist_weight * feature_weight))


Here we can check for the number of connected components and number of isolated nodes in the constructed graph

In [ ]:
isolated_nodes = [node for node, degree in G.degree() if degree == 0]
print("There are", len(isolated_nodes), "isolated nodes")
#G.remove_nodes_from(isolated_nodes)

connected_components = list(nx.connected_components(G))
print("There are", len(connected_components), "connected components")

print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges()) 

Now we can perform normalized cuts and save the cut graph

In [ ]:
import cProfile
import re
#cut_graph = cProfile.run('normalized_cut(G, thresh = 0.01, num_cuts = 10)')
cut_graph = normalized_cut(G, thresh = 0.01, num_cuts = 10)

Now we visualize the different components and save them to a point cloud file

In [ ]:
connected_components = list(nx.connected_components(cut_graph))
print("Number of cut regions:",len(connected_components))

# Visualize the connected components
component_lengths = []
labels = np.zeros(num_points, dtype=np.int32)
for i in range(len(connected_components)):
    labels[list(connected_components[i])] = i
    component_lengths.append(len(connected_components[i]))

print("Component lengths:", component_lengths)

colorspace = plt.cm.rainbow(np.linspace(0, 1, len(set(labels))))[:, :3]
np.random.shuffle(colorspace) #Otherwise close subgraphs look very similar!
colors = [colorspace[i] for i in labels]
pcd.colors = o3d.utility.Vector3dVector(np.array(colors))

#o3d.io.write_point_cloud("pcd_merge_clustered.pcd", pcd, write_ascii=False, compressed=False, print_progress=False)

With this method, we can perform the nearest point labeling on the big point cloud, if the clustering beforehand was done on a sampled point cloud

In [ ]:
pcd_cut.paint_uniform_color([0, 0, 0])
pcd_tree = o3d.geometry.KDTreeFlann(pcd)

i=0
for point in np.asarray(pcd_cut.points):
    [_, idx, _] = pcd_tree.search_knn_vector_3d(point, 1)
    np.asarray(pcd_cut.colors)[i,:] = np.asarray(pcd.colors)[idx[0], :]
    i+=1

o3d.io.write_point_cloud("pcd_merge_clustered.pcd", pcd_cut, write_ascii=False, compressed=False, print_progress=False)

We can also perform spectral clustering instead of normalized cuts:

In [ ]:
n_clusters = 55
A = nx.adjacency_matrix(G).toarray()
sc = SpectralClustering(n_clusters=n_clusters, affinity="precomputed", n_init=100, assign_labels="discretize")
labels = sc.fit_predict(A)  

In [ ]:
colorspace = plt.cm.rainbow(np.linspace(0, 1, n_clusters))[:, :3]
colors = [colorspace[i] for i in labels]
pcd.colors = o3d.utility.Vector3dVector(np.array(colors))
#o3d.io.write_point_cloud("pcd_merge_clustered.pcd", pcd, write_ascii=False, compressed=False, print_progress=False)

Again, we can perform the nearest point labeling on the big point cloud, if the clustering beforehand was done on a sampled point cloud

In [ ]:
pcd_cut.paint_uniform_color([0, 0, 0])
pcd_tree = o3d.geometry.KDTreeFlann(pcd)

i=0
for point in np.asarray(pcd_cut.points):
    [_, idx, _] = pcd_tree.search_knn_vector_3d(point, 1)
    np.asarray(pcd_cut.colors)[i,:] = np.asarray(pcd.colors)[idx[0], :]
    i+=1

o3d.io.write_point_cloud("pcd_merge_clustered.pcd", pcd_cut, write_ascii=False, compressed=False, print_progress=False)

We can also perform other clustering methods of the feature matrix, that do not take the graph as input:

In [ ]:
n_components = 25
tSVD = TruncatedSVD(n_components=25)
transformed_data = tSVD.fit_transform(association_matrix)

In [ ]:
# Birch is fastest, results are also quite nice
birch_model = Birch(threshold=0.1, n_clusters=25)
birch_model.fit(transformed_data)
labels = birch_model.predict(transformed_data)

# kmeans is quite fast, results look okay, some noise
#kmeans = KMeans(n_clusters=80)
#labels = kmeans.fit_predict(transformed_data)

# DBSCAN doesn't really produce great results and takes really long
#dbscan = DBSCAN(eps=0.5, min_samples=100)
#labels = dbscan.fit_predict(transformed_data)

# Meanshift takes way too long
#meanshift = MeanShift()
#labels = meanshift.fit_predict(transformed_data)

In [ ]:
print("Cluster Assignments:")
print(len(set(labels)))

In [ ]:
colorspace = plt.cm.rainbow(np.linspace(0, 1, len(set(labels))))[:, :3]
colors = [colorspace[i] for i in labels]
pcd.colors = o3d.utility.Vector3dVector(np.array(colors))
o3d.io.write_point_cloud("pcd_merge_clustered.pcd", pcd, write_ascii=False, compressed=False, print_progress=False)